## This notebook inputs the missing values using KNN

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [2]:
INPUT, OUTPUT = 'output/\CompleteCovid', 'output/\KNNCovidImputed'

In [3]:
df = pd.read_csv(r'output\/CompleteCovid.csv',index_col=0)
df_train = df.loc['2020-01-22':'2020-02-27'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2020-02-28':, df_train.columns] #Selecting test data
df = df_train

In [4]:
df.shape

(1278, 360)

In [5]:
df_test.shape

(519, 360)

In [6]:
for i in range(0,30):
    #Scaled data has zero mean and unit variance
    scaler = preprocessing.StandardScaler()
    scaler.fit(df)
    array = scaler.transform(df)
    array_test = scaler.transform(df_test)
    df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
    df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
    
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    imp.fit(df_scaled)
    df_imputed = imp.transform(df_scaled)
    df_test_imputed = imp.transform(df_test_scaled)
    df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
    df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)
    # Descaling the data
    array_2 = scaler.inverse_transform(df_final)
    df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

    array_test_2 = scaler.inverse_transform(df_test_final)
    df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)
    df_final2.to_csv(OUTPUT+"_"+str(i)+'_training.csv', index = True)
    df_test_final2.to_csv(OUTPUT+"_"+str(i)+'_test.csv', index = True)